In [39]:
from sympy import *

m_a, m_l = symbols('m_a m_l') #mass of actuator, load
x_a, x_l = symbols('x_a x_l') #position of actuator, load
xd_a, xd_l = symbols('\dot{x_a} \dot{x_l}') #position of actuator, load
x1, x2 = symbols('x1 x2') #position of ends of spring
T1, T2 = symbols('T1 T2') #transmission ratios of transmissions before and after spring
k = symbols('k') #spring stiffness

q = MatrixSymbol('q', 2, 1)
qd = MatrixSymbol('\dot{q}', 2, 1)
qdd = MatrixSymbol('\ddot{q}', 2, 1)
def derivative(x):
    x = Matrix([x])
    derivmat = x.jacobian(Matrix([q,qd])) * Matrix([qd,qdd])
    if len(x) == 1:
        return derivmat[0]
    return derivmat

x_a = q[0]
x_l = q[1]
xd_a = derivative(x_a)
xd_l = derivative(x_l)

x1 = T1*x_a
x2 = 1/T2*x_l

# Kinetic energy
K = (m_a*xd_a*xd_a + m_l*xd_l*xd_l)/2

# Potential energy
P = (k*(x1-x2)**2)/2

# Define Lagrangian
Lag = K - P
Lag = simplify(nsimplify(Lag))

display(K, P, Lag)

m_a*\dot{q}[0, 0]**2/2 + m_l*\dot{q}[1, 0]**2/2

k*(T1*q[0, 0] - q[1, 0]/T2)**2/2

(T2**2*(m_a*\dot{q}[0, 0]**2 + m_l*\dot{q}[1, 0]**2) - k*(T1*T2*q[0, 0] - q[1, 0])**2)/(2*T2**2)

In [40]:
# Euler-Lagrangian Equation of Motion (EoM)
EoM_LHS = zeros(2, 1)
for i in range(2):
    partial_dq_ = simplify(diff(Lag, qd[i]))
    partial_q_ = simplify(diff(Lag, q[i]))
    EoM_LHS[i] = derivative(partial_dq_) - partial_q_

display(simplify(EoM_LHS))
display(EoM_LHS)

Matrix([
[(T1*k*(T1*T2*q[0, 0] - q[1, 0]) + T2*m_a*\ddot{q}[0, 0])/T2],
[    -T1*k*q[0, 0]/T2 + m_l*\ddot{q}[1, 0] + k*q[1, 0]/T2**2]])

Matrix([
[-T1*k*(-T1*T2*q[0, 0] + q[1, 0])/T2 + m_a*\ddot{q}[0, 0]],
[  m_l*\ddot{q}[1, 0] - k*(T1*T2*q[0, 0] - q[1, 0])/T2**2]])

In [41]:
M = simplify(EoM_LHS.jacobian(qdd))
Minv = M.inv()
display(M, Minv)

Matrix([
[m_a,   0],
[  0, m_l]])

Matrix([
[1/m_a,     0],
[    0, 1/m_l]])

In [43]:
CG = simplify(EoM_LHS.subs({qdd:Matrix([0,0])}))
display(CG)

Matrix([
[ T1*k*(T1*T2*q[0, 0] - q[1, 0])/T2],
[k*(-T1*T2*q[0, 0] + q[1, 0])/T2**2]])

In [72]:
motorforce = symbols('f')
tau = Matrix([motorforce, 0])

qdd_sim = simplify(Minv*(tau - CG))
display(qdd_sim[0], qdd_sim[1])

(-T1*k*(T1*T2*q[0, 0] - q[1, 0]) + T2*f)/(T2*m_a)

k*(T1*T2*q[0, 0] - q[1, 0])/(T2**2*m_l)